# Computation of isochrones with a local instance of OpenRouteService using online OSM data 

## Installation libraries pyhton

```bash
conda env create -f environment.yml 
conda activate ORS_stations
pip install -r requirements.txt
```

In [1]:
import os

from IPython.display import display
from ipywidgets import IntProgress

import folium
from folium.plugins import MarkerCluster

import overpass

import time
import pandas as pd
import geopandas as gpd
# import fiona as fn
from shapely.geometry import shape, mapping, LineString
from shapely.ops import cascaded_union, unary_union, polygonize
import shapely
import geojson

import matplotlib.pyplot as plt

import requests
import json

from zonal_stats import * # import zonal stats function from python file, get it here: https://gist.github.com/perrygeo/5667173

api_ov = overpass.API(timeout=600)

## Usage

It is needed to donwload the OpenRouteService engine with:

```bash
wget https://github.com/GIScience/openrouteservice/archive/refs/tags/v7.1.0.zip
unzip v7.1.0.zip
```

Moreover, it is neeed to have installed [docker](https://www.docker.com/) and download the OpenStreetMap data of the area of study. This could be done using the [Geofabrick](https://download.geofabrik.de/) website.


The file should be in the format similar to `italy-latest.osm.pbf`, the one used in this demo. Directly available at the [link](https://download.geofabrik.de/europe/italy.html), or at the [link](https://download.geofabrik.de/europe/italy-latest.osm.pbf). 

The file size is around 2 GB, the download speed depends on your connection. 

Copy the file inside the folder [openrouteservice-7.1.0\docker](openrouteservice-7.1.0\docker).

Then it is need to modify the file [docker-compose.yml](openrouteservice-7.1.0/docker/docker-compose.yml) uncommenting the build option and modify the text to:

```yml
OSM_FILE: ./italy-latest.osm.pbf
```
```yml
- ./italy-latest.osm.pbf:/home/ors/ors-core/data/osm_file.pbf
```

Then opening a terminal launch the following commands:

```bash
cd openrouteservice-7.1.0
cd docker 
mkdir conf elevation_cache graphs logs
cd logs
mkdir ors tomcat 
cd ..
docker compose up
```

At the first starts the streets graphs will be generated. The task required some minutes depending on the file and the machine used.

The creation could be checked at the link [http://localhost:8080/ors/v2/status](http://localhost:8080/ors/v2/status)

## Script

### Initialitation

In [19]:
name_region = "Lombardia"
admin_level = "4" # "2" = State, "4" = Province
charging_station_flag = 1 # 1 = charging stations, 0 = fuel stations

### Download of station from OSM

In [20]:
if charging_station_flag == 1:
    query = "[out:json][timeout:60];area[\"name\"=\"" + name_region + "\"][\"admin_level\"=" + admin_level + "]->.searchArea;node[\"amenity\"=\"charging_station\"](area.searchArea);out center;"
    text_stations = "charging"
else:
    query = "[out:json][timeout:60];area[\"name\"=\"" + name_region + "\"][\"admin_level\"=" + admin_level + "]->.searchArea;node[\"amenity\"=\"fuel\"](area.searchArea);out center;"
    text_stations = "petrol"
stations = api_ov.get(query, build = False)

In [21]:
charging_dict = {}
for feature in stations["elements"]:
    facility_id = int(feature['id'])
    charging_dict[facility_id] = {
        'geometry': {"type": "Point", "coordinates": [ feature["lon"], feature["lat"] ] }
    }
last = [feature["lat"], feature["lon"]]
print("Created dictionary with " + str(len(charging_dict)) + " " + text_stations + " stations")

Created dictionary with 6192 charging stations


In [ ]:
map_outline = folium.Map(tiles='openstreetmap', location=(last), zoom_start=9)

cluster = MarkerCluster().add_to(map_outline)  

for facility_id in charging_dict:
    folium.Marker(list(reversed(charging_dict[facility_id]['geometry']['coordinates']))).add_to(cluster)

map_outline.save(os.path.join('results', "1_" + text_stations + "_station_overview_" + name_region + ".html"))
map_outline

### Computation of isochrones

In [ ]:
headers = {'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8','Content-Type': 'application/json; charset=utf-8'}

i = 0
check = 0
error_counter = 0

iso_car_1 = []
iso_car_5 = []
iso_car_10 = []
list_loc = []

for facility_id in charging_dict.keys():
    loc = charging_dict[facility_id]['geometry']['coordinates']
    list_loc.append(loc)
l = len(list_loc)

In [ ]:
f = IntProgress(min=0, max=l) # instantiate the bar
display(f) # display the bar

while i < l:
        iso_params = {'locations': [list_loc[i]],
                'range_type': 'distance',
                'range': [1000, 5000, 10000]  # meters
                }
                
        call = requests.post('http://localhost:8080/ors/v2/isochrones/driving-car', json=iso_params, headers=headers)
        request = json.loads(call.text)
        check += 1
        
        try:
                iso_car_1.append(shape(request['features'][0]['geometry']))
                iso_car_5.append(shape(request['features'][1]['geometry']))
                iso_car_10.append(shape(request['features'][2]['geometry']))
                i += 1
        except:
                print("Error in " + text_stations + " stations at location: " + str(list_loc[i]))
                print("Check the error on the map at https://www.openstreetmap.org/#map=19/" + str(list_loc[i][1]) + "/" + str(list_loc[i][0]))
                error_counter += 1
                i += 1

        f.value += 1


print("Have been requested " + str(check) + " " + text_stations + "stations")
if error_counter > 0:
       print("Of which " + str(error_counter) + " ended with an error")

In [ ]:
iso_union_car_1 = cascaded_union(iso_car_1)
iso_union_car_5 = cascaded_union(iso_car_5)
iso_union_car_10 = cascaded_union(iso_car_10)
geojson_iso_1 = shapely.to_geojson(iso_union_car_1)
geojson_iso_5 = shapely.to_geojson(iso_union_car_5)
geojson_iso_10 = shapely.to_geojson(iso_union_car_10)

isochrones_car_filename_1 = "results/iso_union_car_1_" + text_stations + "_" + name_region + ".geojson"
isochrones_car_filename_5 = "results/iso_union_car_5_" + text_stations + "_" + name_region + ".geojson"
isochrones_car_filename_10 = "results/iso_union_car_10_" + text_stations + "_" + name_region + ".geojson"

# save isochrones to geojson
with open(isochrones_car_filename_1, 'w') as f:
    f.write(geojson_iso_1)
with open(isochrones_car_filename_5, 'w') as f:
    f.write(geojson_iso_5)
with open(isochrones_car_filename_10, 'w') as f:
    f.write(geojson_iso_10)

In [ ]:
map_isochrones = folium.Map(tiles='openstreetmap', location=(last), zoom_start=9)

def style_function(color):  # To style isochrones
    return lambda feature: dict(color=color)

folium.GeoJson(geojson_iso_10, name="10 km", style_function = style_function('#fde725ff')).add_to(map_isochrones)
folium.GeoJson(geojson_iso_5, name="5 km",style_function = style_function('#1f968bff')).add_to(map_isochrones)
folium.GeoJson(geojson_iso_1, name="1 km",style_function = style_function('#440154ff')).add_to(map_isochrones)

cluster = MarkerCluster().add_to(map_isochrones)  

for facility_id in charging_dict:
    folium.Marker(list(reversed(charging_dict[facility_id]['geometry']['coordinates']))).add_to(cluster)

map_isochrones.save(os.path.join('results', "2_"  + text_stations + "_isochrones_" + name_region + ".html"))
map_isochrones

### Download of highways from OSM

In [ ]:
query_2 = "[out:json][maxsize:1000000000][timeout:600];area[\"name\"=\"" + name_region + "\"][\"admin_level\"=" + admin_level +"]->.searchArea;(way[\"highway\"=\"motorway\"](area.searchArea);way[\"highway\"=\"trunk\"](area.searchArea);way[\"highway\"=\"primary\"](area.searchArea);way[\"highway\"=\"secondary\"](area.searchArea);way[\"highway\"=\"tertiary\"](area.searchArea);way[\"highway\"=\"motorway_link\"](area.searchArea);way[\"highway\"=\"trunk_link\"](area.searchArea);way[\"highway\"=\"primary_link\"](area.searchArea);way[\"highway\"=\"secondary_link\"](area.searchArea);way[\"highway\"=\"tertiary_link\"](area.searchArea);way[\"highway\"=\"unclassified\"](area.searchArea);way[\"highway\"=\"residential\"](area.searchArea);way[\"highway\"=\"living_street\"](area.searchArea);way[\"highway\"=\"service\"](area.searchArea););out geom;"

highways = api_ov.get(query_2, build = False)

strade_dict = [{
    'id': element['id'],
    'class': element['tags']['highway'],
    'geometry': LineString([(coords['lon'], coords['lat']) for coords in element['geometry']]),
} for element in highways['elements']]

highways_region = gpd.GeoDataFrame(strade_dict)
highways_region = highways_region.set_crs('epsg:4326')
highways_region.to_file("results/highways_"+ name_region + ".geojson", driver="GeoJSON")  

### Intersection between isochrones and highways

In [ ]:
iso_car_1_geo = gpd.read_file("results/iso_union_car_1_" + text_stations + "_" + name_region + ".geojson", driver="GeoJSON")
iso_car_5_geo = gpd.read_file("results/iso_union_car_5_" + text_stations + "_" + name_region + ".geojson", driver="GeoJSON")
iso_car_10_geo = gpd.read_file("results/iso_union_car_10_" + text_stations + "_" + name_region + ".geojson", driver="GeoJSON")

strade_iso_car_1 = highways_region[highways_region.intersects(iso_car_1_geo.geometry.iloc[0])]
strade_iso_car_5 = highways_region[highways_region.intersects(iso_car_5_geo.geometry.iloc[0])]
strade_iso_car_10 = highways_region[highways_region.intersects(iso_car_10_geo.geometry.iloc[0])]

In [ ]:
strade_iso_car_1.to_file("results/strade_iso_car_1_" + text_stations + "_" + name_region + ".geojson", driver="GeoJSON")  
strade_iso_car_5.to_file("results/strade_iso_car_5_" + text_stations + "_" + name_region + ".geojson", driver="GeoJSON")  
strade_iso_car_10.to_file("results/strade_iso_car_10_" + text_stations + "_" + name_region + ".geojson", driver="GeoJSON")  

In [ ]:
map_streets = folium.Map(tiles='openstreetmap', location=(last), zoom_start=9)

folium.GeoJson(highways_region.to_json(), name=">10 km", style_function = style_function('#F08080')).add_to(map_streets)
folium.GeoJson(strade_iso_car_10.to_json(), name="10 km", style_function = style_function('#fde725ff')).add_to(map_streets)
folium.GeoJson(strade_iso_car_5.to_json(), name="5 km",style_function = style_function('#1f968bff')).add_to(map_streets)
folium.GeoJson(strade_iso_car_1.to_json(), name="1 km",style_function = style_function('#440154ff')).add_to(map_streets)

map_streets.save(os.path.join('results', "3_streets_isocrones" + text_stations + "_" + name_region + ".html"))
# The date in the map are significant and the load of the map could slow al lot the results
# it is suggested to open the html in a broser
# map_streets

### Results

In [ ]:
l_1 = strade_iso_car_1.to_crs('epsg:3857').geometry.length.sum()
l_5 = strade_iso_car_5.to_crs('epsg:3857').geometry.length.sum() - l_1
l_10 = strade_iso_car_10.to_crs('epsg:3857').geometry.length.sum() - l_5 - l_1
l_region = highways_region.to_crs('epsg:3857').geometry.length.sum()
l_over_10 = l_region - l_10 - l_5 - l_1

data = {
    "Distanza da colonnina": ["< 1 km", "1 - 5 km", "5 - 10 km", "> 10 km"],
    "Lunghezza totale (km)": [l_1/1000, l_5/1000, l_10/1000, l_over_10/1000],
    "Percentuale classe (%)": [l_1/l_region*100, l_5/l_region*100, l_10/l_region*100, l_over_10/l_region*100]
}

pd.DataFrame(data)

In [ ]:
final_distribution = pd.DataFrame(data)
final_distribution.to_csv("results/distribution_" + text_stations + "_" + name_region + ".csv")